In [283]:
import os

n_cpu = os.cpu_count()

In [284]:
import pandas as pd
import numpy as np

In [285]:
dummy_data = pd.read_csv("apolipo.csv")
dummy_data.shape            # 421 attributes

(188, 421)

In [286]:
dummy_data.describe()

,A,C,D,E,F,G,H,I,K,L,...,YN,YP,YQ,YR,YS,YT,YV,YW,YY,y
count,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,...,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000
mean,0.077575,0.030471,0.041081,0.054918,0.040699,0.074569,0.024076,0.041579,0.049528,0.108409,...,0.001473,0.001701,0.000996,0.001902,0.002197,0.001525,0.001614,0.000631,0.001008,0.281915
std,0.032832,0.028688,0.016485,0.024134,0.021358,0.035840,0.016050,0.024393,0.026055,0.034274,...,0.002703,0.002541,0.001863,0.003148,0.002927,0.002358,0.002166,0.001629,0.001860,0.451134
min,0.025000,0.000000,0.000000,0.006361,0.000000,0.007634,0.000000,0.000000,0.005597,0.022959,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.054178,0.009298,0.029169,0.038685,0.027805,0.055556,0.015311,0.024633,0.030773,0.085733,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.072688,0.022185,0.040906,0.052982,0.035923,0.070618,0.022011,0.037054,0.049268,0.105646,...,0.000000,0.000000,0.000000,0.000000,0.000958,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.094954,0.044164,0.053207,0.064842,0.050065,0.087584,0.030410,0.053343,0.064068,0.128123,...,0.001936,0.002778,0.001697,0.002785,0.003770,0.002494,0.002694,0.000000,0.001704,1.000000
max,0.212468,0.137097,0.087049,0.157724,0.118451,0.284987,0.137255,0.147510,0.125000,0.226562,...,0.014925,0.012821,0.012821,0.020000,0.014925,0.012821,0.010870,0.010000,0.008475,1.000000


In [287]:
dummy_data['y'].value_counts()

0    135
1     53
Name: y, dtype: int64

In [288]:
X = dummy_data.drop(columns='y')
y = dummy_data['y']

In [289]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)

In [290]:
max_population = 30
max_iteration = 50

np.random.seed(42)

In [291]:
### jaya algo ancillary functions ###

In [292]:
population = []
population_member = []

for i in range(max_population):
    population_member = np.random.uniform(low=0.0, high=1.0, size=X.shape[1])
    population.append(population_member)

# initialising the first population, RANDOMLY

In [293]:
# in the above cell, uniform creates an even distribution for the values it is picking from
# X.shape[1] is the number of attributes present
# the above cell is creating a random population of attributes to be used in the algorithm

In [294]:
def clean_population(population):

    for i in range(max_population):
        if all(population[i]) < 0.5:
            population[i] = np.random.uniform(low=0.0, high=1.0, size=X.shape[1])

    for i in range(max_population):
        if all(population[i]) < 0.5:
            population[i] = np.random.uniform(low=0.0, high=1.0, size=X.shape[1])

    return population

In [295]:
# the above code is making sure there is at least one attribute to be tested for that iteration
# doing it twice is just completely make sure

# 0.5 is the threshold to take that attribute or not,,, it is arbitrarily chosen
# in this case to give an even chance for the value to be chosen or not in that population_member

In [296]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score

def scoring_curr_model(population):
    
    scores = []
    population = clean_population(population)

    for i in range(max_population):
        drop_columns = []
        for j in range(X.shape[1]):
                if population[i][j] < 0.5:
                    drop_columns.append((X.columns[[j]])[0])
        X_new = X.drop(columns=drop_columns)                    # ask him how he did it one line
        y_new = y
        
        cv_score = cross_val_score(rf_model, X_new, y_new, scoring=make_scorer(accuracy_score), cv=5, n_jobs=n_cpu-1)
        cv_score_mean = np.mean(cv_score)
        scores.append(cv_score_mean)
    
    return scores

In [297]:
### Simple JAYA Algorithm ###

for i in range(max_iteration):

    curr_cv_scores = scoring_curr_model(population)
    best_score = max(curr_cv_scores)
    worst_score = min(curr_cv_scores)

    best_member = population[curr_cv_scores.index(best_score)]
    worst_member = population[curr_cv_scores.index(worst_score)]


    for j in range(max_population):
        population[j] = population[j] + np.random.uniform(0,1)*(population[j] - best_member) - np.random.uniform(0,1)*(population[j] - worst_member)
        for k in range(X.shape[1]):
            if population[j][k] < 0:
                population[j][k] = 0
            elif population[j][k] > 1:
                population[j][k] = 1


In [298]:
# Attributes:
drop_columns = []
for i in range(X.shape[1]):
        if best_member[i] < 0.5:
            drop_columns.append((X.columns[[i]])[0])
attribute = X.drop(columns=drop_columns)
attribute.columns

Index(['A', 'C', 'D', 'F', 'G', 'I', 'P', 'S', 'V', 'W',
       ...
       'YC', 'YD', 'YG', 'YI', 'YK', 'YL', 'YM', 'YN', 'YR', 'YW'],
      dtype='object', length=207)

In [299]:
# Score:
best_score

# the highest score i got was 0.90967 (with 42 as random state everywhere)

0.9096728307254622